In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website---------------------------------------------------------------------------
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

# Initialize Driver----------------------------------------------------------------------------
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver


def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes----------------------------------------------------------------
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details---------------------------------------------------------------
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" Button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details---------------------------------------------------------------------
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages-------------------------------------------------------------
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame------------------------------------------------------
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-chandigarh: Message: script timeout
  (Session info: chrome=129.0.6668.58)
Stacktrace:
	GetHandleVerifier [0x00007FF63766FDA5+29557]
	(No symbol) [0x00007FF6375E2240]
	(No symbol) [0x00007FF63749B519]
	(No symbol) [0x00007FF637539BB6]
	(No symbol) [0x00007FF6375174FA]
	(No symbol) [0x00007FF6375388A3]
	(No symbol) [0x00007FF6375172A3]
	(No symbol) [0x00007FF6374E12DF]
	(No symbol) [0x00007FF6374E2451]
	GetHandleVerifier [0x00007FF63799DCBD+3363469]
	GetHandleVerifier [0x00007FF6379E9B47+3674391]
	GetHandleVerifier [0x00007FF6379DEAEB+3629243]
	GetHandleVerifier [0x00007FF63772FC66+815670]
	(No symbol) [0x00007FF6375ED6EF]
	(No symbol) [0x00007FF6375E92B4]
	(No symbol) [0x00007FF6375E9450]
	(No symbol) [0x00007FF6375D81FF]
	BaseThreadInitThunk [0x00007FFBC57C257D+29]
	RtlUserThreadStart [0x00007FFBC702AF28+40]



NameError: name 'driver' is not defined

In [6]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165754,HVAC Seater (2+3),06:20,02h 45m,09:05,4.2,INR 195.6,39 Seats available
1,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165750,HVAC Seater (2+3),12:20,02h 45m,15:05,3.9,INR 195.6,47 Seats available
2,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165752,HVAC Seater (2+3),14:20,02h 45m,17:05,4.0,INR 195.6,45 Seats available
3,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Zimindara Travels,AC Sleeper (2+1),20:30,02h 00m,22:30,4.0,INR 699,36 Seats available
4,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:40,02h 50m,23:30,3.8,INR 599,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
1031,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),14:13,04h 05m,18:18,3.8,INR 650,45 Seats available
1032,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),15:25,04h 05m,19:30,3.6,INR 650,39 Seats available
1033,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Jujhar Travels,Volvo A/C Semi Sleeper (2+2),17:25,03h 55m,21:20,4.3,INR 650,45 Seats available
1034,Chandigarh to Firozpur,https://www.redbus.in/bus-tickets/chandigarh-t...,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),14:55,04h 05m,19:00,2.7,INR 666,45 Seats available
